In [2]:
import torch
import os
import pickle
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
DIR = '../../embeddings2/Visualfeatures/'
files = os.listdir(DIR)
embeddings_map = {}
for file in tqdm(files):
    with open(DIR + file, 'rb') as f:
        key = file.split('.')[0]
        key = '-'.join(key.split('-')[:3])
        embeddings_map[key] = pickle.load(f).squeeze()
with open('../../embeddings2/visual_features.pkl', 'wb') as f:
    pickle.dump(embeddings_map, f)

100%|██████████| 149/149 [00:00<00:00, 2328.26it/s]


In [3]:
DIR = '../../embeddings2/DeepIDfeatures_cache2/'
files = os.listdir(DIR)
embeddings_map = {}
for file in tqdm(files):
    with open(DIR + file, 'rb') as f:
        key = file.split('.')[0]
        key = '-'.join(key.split('-')[:3])
        embeddings_map[key] = torch.mean(pickle.load(f).squeeze(), dim=0).float()
with open('../../embeddings2/visual_features_2.pkl', 'wb') as f:
    pickle.dump(embeddings_map, f)

100%|██████████| 149/149 [00:01<00:00, 116.42it/s]


In [4]:
DIR = '../../embeddings2/Facenetfeatures_cache2/'
files = os.listdir(DIR)
embeddings_map = {}
for file in tqdm(files):
    with open(DIR + file, 'rb') as f:
        key = file.split('.')[0]
        key = '-'.join(key.split('-')[:3])
        embeddings_map[key] = torch.mean(pickle.load(f).squeeze(), dim=0).float()
with open('../../embeddings2/visual_features_3.pkl', 'wb') as f:
    pickle.dump(embeddings_map, f)

100%|██████████| 149/149 [00:01<00:00, 141.46it/s]


In [5]:
DIR = '../../embeddings2/OpenFacefeatures_cache2/'
files = os.listdir(DIR)
embeddings_map = {}
for file in tqdm(files):
    with open(DIR + file, 'rb') as f:
        key = file.split('.')[0]
        key = '-'.join(key.split('-')[:3])
        embeddings_map[key] = torch.mean(pickle.load(f).squeeze(), dim=0).float()
with open('../../embeddings2/visual_features_4.pkl', 'wb') as f:
    pickle.dump(embeddings_map, f)

100%|██████████| 149/149 [00:01<00:00, 108.72it/s]


In [7]:
DIR = '../../embeddings2/VGG-Facefeatures_cache2/'
files = os.listdir(DIR)
embeddings_map = {}
for file in tqdm(files):
    with open(DIR + file, 'rb') as f:
        key = file.split('.')[0]
        key = '-'.join(key.split('-')[:3])
        embeddings_map[key] = torch.mean(pickle.load(f).squeeze(), dim=0).float()
with open('../../embeddings2/visual_features_5.pkl', 'wb') as f:
    pickle.dump(embeddings_map, f)

100%|██████████| 149/149 [00:01<00:00, 78.91it/s]


In [9]:
class VisualClassifier(torch.nn.Module):
    def __init__(self, embedding_size):
        super(VisualClassifier, self).__init__()
        self.linear1 = torch.nn.Linear(embedding_size, 64)
        self.linear2 = torch.nn.Linear(64, 2)
        # Xavier initialize the linear layer
        torch.nn.init.kaiming_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
    
    def forward(self, x):
        x = self.linear1(torch.nn.functional.relu(x))
        return self.linear2(torch.nn.functional.relu(x))

In [10]:
class VisualClassifierPCA(torch.nn.Module):
    def __init__(self, embedding_size):
        super(VisualClassifierPCA, self).__init__()
        self.linear1 = torch.nn.Linear(embedding_size, 16)
        self.linear2 = torch.nn.Linear(16, 2)
        # Xavier initialize the linear layer
        torch.nn.init.kaiming_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
    
    def forward(self, x):
        x = self.linear1(torch.nn.functional.relu(x))
        return self.linear2(torch.nn.functional.relu(x))

In [11]:
class VisualDataset(Dataset):
    def __init__(self, data, embeddings_file):
        with open(embeddings_file, 'rb') as f:
            embeddings_map = pickle.load(f)
        self.labels = [x[1] for x in data]
        data = [x[0] for x in data]
        self.data = [embeddings_map[file] for file in data]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index], self.labels[index]

In [12]:
class VisualDatasetPCA(Dataset):
    def __init__(self, data, embeddings_file, pca, sc, test=False):
        with open(embeddings_file, 'rb') as f:
            embeddings_map = pickle.load(f)
        self.labels = [x[1] for x in data]
        data = [x[0] for x in data]
        self.data = np.array([embeddings_map[file].numpy() for file in data])
        if not test:
            self.data = sc.fit_transform(self.data)
            self.data = pca.fit_transform(self.data)
        else:
            self.data = sc.transform(self.data)
            self.data = pca.transform(self.data)
        self.data = torch.from_numpy(self.data).float()
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index], self.labels[index]

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
def train(model, train_loader, criterion, optimizer, num_epochs):
    for _ in range(num_epochs):
        model.train()
        for x, y in train_loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            loss = criterion(out, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [15]:
def eval(model, val_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            _, predicted = torch.max(out.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
        return correct / total

In [16]:
train_data = []
for file in os.listdir('../../data/Clips/Truthful/'):
    train_data.append((file.split('.')[0], 0))
for file in os.listdir('../../data/Clips/Deceptive/'):
    train_data.append((file.split('.')[0], 1))

test_data = []
for file in os.listdir('../../Dataset2/Videos/Truth/'):
    key = file.split('.')[0]
    key = '-'.join(key.split('-')[:3])
    test_data.append((key, 0))
for file in os.listdir('../../Dataset2/Videos/Lie/'):
    key = file.split('.')[0]
    key = '-'.join(key.split('-')[:3])
    test_data.append((key, 1))

In [17]:
def get_accuracy(train_embedding_file, test_embedding_file, do_pca, embedding_size, train_batch_size, num_epochs):
    if not do_pca:
        train_dataset = VisualDataset(train_data, train_embedding_file)
        test_dataset = VisualDataset(test_data, test_embedding_file)
        model = VisualClassifier(embedding_size).to(device)
    else:
        pca = PCA(n_components=embedding_size)
        sc = StandardScaler()
        train_dataset = VisualDatasetPCA(train_data, train_embedding_file, pca, sc, False)
        test_dataset = VisualDatasetPCA(test_data, test_embedding_file, pca, sc, True)
        model = VisualClassifierPCA(embedding_size).to(device)
    train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    learning_rate = 1e-3
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train(model, train_loader, criterion, optimizer, num_epochs)
    return eval(model, test_loader)

In [19]:
# Model accuracy for resnet-3d-18
get_accuracy('../../embeddings/visual_features.pkl', '../../embeddings2/visual_features.pkl', False, 512, 4, 40)

0.4966442953020134

In [22]:
# Model accuracy for resnet-3d-18 with pca
get_accuracy('../../embeddings/visual_features.pkl', '../../embeddings2/visual_features.pkl', True, 64, 4, 40)

0.42953020134228187

In [18]:
# Model accuracy for deep id
get_accuracy('../../embeddings/visual_features_2.pkl', '../../embeddings2/visual_features_2.pkl', False, 160, 4, 40)

0.4899328859060403

In [22]:
# Model accuracy for deep id with pca
get_accuracy('../../embeddings/visual_features_2.pkl', '../../embeddings2/visual_features_2.pkl', True, 64, 4, 40)

0.4966442953020134

In [19]:
# Model accuracy for facenet
get_accuracy('../../embeddings/visual_features_3.pkl', '../../embeddings2/visual_features_3.pkl', False, 128, 4, 40)

0.4899328859060403

In [23]:
# Model accuracy for facenet with pca
get_accuracy('../../embeddings/visual_features_3.pkl', '../../embeddings2/visual_features_3.pkl', True, 64, 4, 40)

0.5100671140939598

In [20]:
# Model accuracy for openface
get_accuracy('../../embeddings/visual_features_4.pkl', '../../embeddings2/visual_features_4.pkl', False, 128, 4, 40)

0.4228187919463087

In [24]:
# Model accuracy for openface with pca
get_accuracy('../../embeddings/visual_features_4.pkl', '../../embeddings2/visual_features_4.pkl', True, 64, 4, 40)

0.47651006711409394

In [21]:
# Model accuracy for vggface
get_accuracy('../../embeddings/visual_features_5.pkl', '../../embeddings2/visual_features_5.pkl', False, 2622, 4, 40)

0.46308724832214765

In [25]:
# Model accuracy for vggface with pca
get_accuracy('../../embeddings/visual_features_5.pkl', '../../embeddings2/visual_features_5.pkl', True, 64, 4, 40)

0.4899328859060403